In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import janitor

In [10]:
iija_df = pd.read_excel("iija.xlsx").clean_names()
iija_df.head()

,state_teritory_or_tribal_nation,total_billions_
0,ALABAMA,3.0000
1,ALASKA,3.7000
2,AMERICAN SAMOA,0.0686
3,ARIZONA,3.5000
4,ARKANSAS,2.8000


https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population


In [34]:
import re
import requests
from io import StringIO

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population"

# Send request
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(url, headers=headers)
response.raise_for_status()

# Read the first table from the page
df = pd.read_html(StringIO(response.text))[0]

# Flatten multi-level columns and normalize names
df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
df.columns = [re.sub(r'\[.*?\]', '', col.lower().replace(' ', '_').replace('\n', '_')).strip('_') for col in df.columns]

# Keep only the first two columns and rename
df = df.iloc[:56, :2]
df.columns = ['state', 'population']

# Clean 'state' column and convert 'population' to integer
df['state'] = df['state'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['population'] = df['population'].astype(int)

In [35]:
df

,state,population
0,California,39431263
1,Texas,31290831
2,Florida,23372215
3,New York,19867248
4,Pennsylvania,13078751
5,Illinois,12710158
6,Ohio,11883304
7,Georgia,11180878
8,North Carolina,11046024
9,Michigan,10140459


In [ ]:
df.to_csv("us_population_by_state_clean.csv", index=False)